In [32]:
import pandas as pd

df = pd.read_csv('2023-03-27_steam_top_5000.csv')

df.shape

(5000, 17)

## 1. 특성 정리(전처리)
1. `score_rank`, `userscore`는 있는 게임이 거의 없으므로 제거한다.
2. `owners` : API에서 정확한 게임의 보유자 수를 제공하지 않고 범주형으로 제공하므로 범주형으로 처리한다
    - 조금 더 보기 간편하라고 단위를 m(ilion), k(ilo)로 바꿔서 넣음
    - `over 2m` 이랑 `2m ~ 5m` 중 뭐가 더 보기 좋은건지는 모르겠다
    - 범주형으로 바꾸고 정렬함(Tableau에서 다시 정렬해야 함)
3. 평가 수 -`evaluation_count`
    - 3-1. 중심극한의 정리 : 표본이 적당히 크다면(>30) 표본평균의 평균이 이루는 분포는 정규분포를 따른다.
        - 좋싫비에 들어가는 값 때문에 고민했던 내용인데, 평가 수가 30개 이상인 게임들만을 데이터프레임에 남겨놓겠음
4. 좋아요&싫어요 비율 - `good_ratio` : $\frac{positive}{positive+negative}$
5. `free` : 0과 1로 구성되어 있으며, `initial_price`가 0이면 1, 아니면 0
6. `median_owners` : `owners`의 가운뎃값을 수치형 데이터로 얻음
7. `count_owner_ratio` : $\frac{evaulation_counts}{MedianOwners}$


In [33]:
# 1.
df = (df.drop(['score_rank', 'userscore'], axis = 1) # 필요 없는 데이터 제거
       .dropna() # publisher, developer에 있는 null값을 axis 기준으로 제거
     )

# 2. 
df['owners'].unique()
df['owners'].replace({'200,000,000 .. 500,000,000' : '200m ~ 500m',
                      '100,000,000 .. 200,000,000' : '100m ~ 200m',
                     '50,000,000 .. 100,000,000' : '50m ~ 100m',
                     '20,000,000 .. 50,000,000' : '20m ~ 50m',
                     '10,000,000 .. 20,000,000' : '10m ~ 20m',
                     '5,000,000 .. 10,000,000' : '5m ~ 10m',
                     '2,000,000 .. 5,000,000': '2m ~ 5m',
                     '1,000,000 .. 2,000,000' : '1m ~ 2m',
                     '500,000 .. 1,000,000' : '500k ~ 1m',
                     '200,000 .. 500,000' : '200k ~ 500k',
                     '100,000 .. 200,000' : '100k ~ 200k'}, inplace = True)

# object -> Categorical로 바꾸고 수동 sorting (Tableau에는 적용 안되겠지..?)
owners_order = pd.CategoricalDtype(['200m ~ 500m', '100m ~ 200m', '50m ~ 100m', 
                                    '20m ~ 50m', '10m ~ 20m', '5m ~ 10m',
                                   '2m ~ 5m', '1m ~ 2m', '500k ~ 1m', '200k ~ 500k',
                                   '100k ~ 200k'])
df['owners'] = df['owners'].astype(owners_order)
# df.sort_values('owners')

# 3. 
df['evaluation_count'] = df['positive'] + df['negative']
# 3-1.
df.drop(df[df['evaluation_count'] < 30].index, inplace = True)

# 4.
df['good_ratio'] = df['positive'] / df['evaluation_count']
df.dropna(inplace = True)

# 5. 
df.loc[df['initialprice'] == 0, 'free'] = 'free'
df.loc[df['initialprice'] != 0, 'free'] = 'paid'

# 6. 
def get_owners_median(x):
    # 이건 노가다밖에 답이 없는 것 같다!
    if x == '200m ~ 500m':
        return int(350 * 1e6)
    elif x == '100m ~ 200m':
        return int(150 * 1e6)
    elif x == '50m ~ 100m':
        return int(75 * 1e6)
    elif x == '20m ~ 50m':
        return int(35 * 1e6)
    elif x == '10m ~ 20m':
        return int(15 * 1e6)
    elif x == '5m ~ 10m':
        return int(7.5 * 1e6)
    elif x == '2m ~ 5m':
        return int(3.5 * 1e6)
    elif x == '1m ~ 2m':
        return int(1.5 * 1e6)
    elif x == '500k ~ 1m':
        return int(750 * 1e3)
    elif x == '200k ~ 500k':
        return int(350 * 1e3)
    elif x == '100k ~ 200k':
        return int(150 * 1e3)
    
df['median_owners'] = df['owners'].apply(get_owners_median).astype("int64")

# 7.
df['count_owner_ratio'] = df['evaluation_count'] / df['median_owners']


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4849 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   appid              4849 non-null   int64   
 1   name               4849 non-null   object  
 2   developer          4849 non-null   object  
 3   publisher          4849 non-null   object  
 4   positive           4849 non-null   int64   
 5   negative           4849 non-null   int64   
 6   owners             4849 non-null   category
 7   average_forever    4849 non-null   int64   
 8   average_2weeks     4849 non-null   int64   
 9   median_forever     4849 non-null   int64   
 10  median_2weeks      4849 non-null   int64   
 11  price              4849 non-null   int64   
 12  initialprice       4849 non-null   int64   
 13  discount           4849 non-null   int64   
 14  ccu                4849 non-null   int64   
 15  evaluation_count   4849 non-null   int64   
 16  good_r

In [36]:
df.to_csv('2023-03-27_steam_top_5000_cleaned.csv')

# 데이터 탐색(with Tableau)

## 1. 보유자 수보다 CCU가 더 중요하다
![image](image2.PNG)

1. 게임을 가지고 있다고 해서 꼭 플레이하고 있는 것은 아니다. 가진 사람이 많다고 플레이하는 사람이 꼭 많지는 않다.
    - `New world`처럼 **보유자 수에 비해 현재 플레이하고 있는 인원이 극도로 적은 케이스**가 있다.
    - `Dota 2`의 보유자 수가 가장 많지만, CCU가 가장 많은 게임은 `글옵(카운터 스트라이크 : 글로벌 오펜시브)`이다.
2. 보유자 수가 많은 게임들은 무료 게임이 많다
3. 그러나 CCU 측면에서도 독보적인 **상위 4개의 작품을 제외하면 게임의 유료 / 무료 여부에 CCU가 정해진다고 보기는 힘들다.**

## 2. 유료 게임의 좋아요 비율과 평가 비율이 더 높다
![image](image3.PNG)

1. **유료 게임**에서 좋아요 / 싫어요 비율이 더 높고, 게임을 평가한 사람의 비율(0.7% vs 1.25%)도 더 높다.
2. 또한, 보유자 수가 많을수록 **좋싫비와 평가자 수가 증가하는 경향**도 보인다.
3. 특이한 점이 두 곳 눈에 띈다.
    1. 왼쪽의 유료 게임 `50m ~ 100m`에서 좋싫비가 급감하는 지점.
    2. 오른쪽의 무료 `50m ~ 100m`에서 평가자 수가 급등하는 지점.
    - 이들을 파악하기 위해선 더 자세하게 들여다볼 필요가 있겠다.